In [162]:
import pandas as pd

def remap(df, collum, j=0):
    new_values = {}
    for i in df[collum].unique():
        new_values[i] = j
        j+=1
    return df[collum].map(new_values)


df = pd.read_csv("./Hotel_Reservations.csv", sep=',')
# print(df)
print(df["type_of_meal_plan"].unique())
meals = {"Not Selected":0, "Meal Plan 1": 1, "Meal Plan 2":2, "Meal Plan 3": 3};
df["type_of_meal_plan"] = df["type_of_meal_plan"].map(meals)
print(df["type_of_meal_plan"])
df["room_type_reserved"]=remap(df, "room_type_reserved", j=1)
# print(df["room_type_reserved"])
df["market_segment_type"] = remap(df, "market_segment_type")
df["booking_status"] = remap(df, "booking_status")
df.drop('Booking_ID', axis='columns', inplace=True)
# df.to_csv("./Hotel_Reservations_Treated.csv")
print((df["booking_status"].sum()/df.shape[0]))

['Meal Plan 1' 'Not Selected' 'Meal Plan 2' 'Meal Plan 3']
0        1
1        0
2        1
3        1
4        0
        ..
36270    1
36271    1
36272    1
36273    0
36274    1
Name: type_of_meal_plan, Length: 36275, dtype: int64
0.32763611302549966


In [176]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

Y = df['booking_status'].values
# X = df.drop(columns=['booking_status', 'no_of_previous_bookings_not_canceled', 'no_of_previous_cancellations', 'arrival_month', 'arrival_date', 'avg_price_per_room'])
X = df[['lead_time','no_of_previous_bookings_not_canceled', 'no_of_previous_cancellations', 'repeated_guest', 'avg_price_per_room']]
numeroFolds = 5
kf = KFold(numeroFolds, shuffle=True)

knn2 = KNeighborsClassifier()
#create a dictionary of all values we want to test for n_neighbors
param_grid = {'n_neighbors': np.arange(1, 25)}
#use gridsearch to test all values for n_neighbors
knn_gscv = GridSearchCV(knn2, param_grid, cv=kf)
#fit model to data
X_train_gscv, _, Y_train_gscv, _ = train_test_split(X, Y, test_size=0.80, random_state=23, stratify=Y)
knn_gscv.fit(X_train_gscv, Y_train_gscv)
print(knn_gscv.best_params_, knn_gscv.best_score_)
nneighbors=int(knn_gscv.best_params_['n_neighbors'])

{'n_neighbors': 4} 0.7871812543073743


In [164]:



# print(df.shape)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=23, stratify=Y)

print(Y_train.sum()/Y_train.shape[0])
knn = KNeighborsClassifier(n_neighbors=nneighbors)
knn.fit(X_train, Y_train);
knn.predict(X_test)
print(knn.score(X_test, Y_test))

print(X.shape, Y.shape)
scores = cross_val_score(knn, X, Y, cv=kf)

print('Acurácia com 10 K-NN: %0.4f +/- %0.4f' % (scores.mean(), scores.std()))


0.32764831287216056
0.8093505347888411
(36275, 4) (36275,)
Acurácia com 10 K-NN: 0.8066 +/- 0.0036


In [165]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Predict using the trained KNN model
Y_pred = knn.predict(X_test)

# Calculate accuracy score
accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy:", accuracy)

# Generate confusion matrix
cm = confusion_matrix(Y_test, Y_pred)
print("Confusion matrix:\n", cm)
# Create a dataframe with X_test
X_test_df = pd.DataFrame(X_test, columns=X.columns)

# Add columns for Y_pred and Y_test
X_test_df['Y_pred'] = Y_pred
X_test_df['Y_test'] = Y_test

# Print the dataframe
# print(X_test_df.head())
X_test_df.to_csv("./ComparePredictionKNN.csv")


Accuracy: 0.8093505347888411
Confusion matrix:
 [[5705  393]
 [1336 1635]]


In [177]:
from sklearn.naive_bayes import GaussianNB
numeroFolds = 10
X = X[['lead_time', 'avg_price_per_room']]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=23, stratify=Y)
kf = KFold(numeroFolds, shuffle=True)
nb = GaussianNB()
nb.fit(X_train, Y_train)
scores = cross_val_score(nb, X_train, Y_train, cv=kf)
print('Acurácia com Naive-Bayes: %0.4f +/- %0.4f' % (scores.mean(), scores.std()))

Y_pred = nb.predict(X_test)

# Calculate accuracy score
accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy:", accuracy)

# Generate confusion matrix
cm = confusion_matrix(Y_test, Y_pred)
print("Confusion matrix:\n", cm)
# Create a dataframe with X_test
X_test_df = pd.DataFrame(X_test, columns=X.columns)

# Add columns for Y_pred and Y_test
X_test_df['Y_pred'] = Y_pred
X_test_df['Y_test'] = Y_test

# Print the dataframe
# print(X_test_df.head())
X_test_df.to_csv("./ComparePredictionNB.csv")

Acurácia com Naive-Bayes: 0.7643 +/- 0.0056
Accuracy: 0.7654647700959312
Confusion matrix:
 [[5623  475]
 [1652 1319]]


In [ ]:
from sklearn.linear_model import LogisticRegression

rlog = LogisticRegression(max_iter=1000)
scores = cross_val_score(rlog, X, Y, cv=kf)
print('Acurácia com Regressão Logística: %0.4f +/- %0.4f' % (scores.mean(), scores.std()))